# Import and clean

In [44]:
import numpy as np
import pandas as pd 

In [45]:
path = "./taxi_data/"
yellow_path = "yellow_taxi/yellow_tripdata_2019-"
yellow_fields = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID']

green_path = "green_taxi/green_tripdata_2019-"
green_fields = ['lpep_pickup_datetime','lpep_dropoff_datetime','PULocationID','DOLocationID']

for_hire_path = "for_hire/fhv_tripdata_2019-"
for_hire_fields = ['pickup_datetime','dropoff_datetime','PULocationID','DOLocationID']

high_volume_path = "high_volume_for_hire/fhvhv_tripdata_2019-"
high_volume_fields = ['pickup_datetime','dropoff_datetime','PULocationID','DOLocationID']

headers = ['travel_time',
'pickup_time',
'OrigLat',
'OrigLon',
'DestLat',
'DestLon',
'Distance']



train_filename = 'cleaned.csv'

test_filename ='test.csv'

In [46]:
def info(data):
    print("Rows     : ", data.shape[0])
    print("Columns  : ", data.shape[1])
    print("\nFeatures : \n", data.columns.tolist())
    print("\nMissing values :  ", data.isnull().sum().values.sum())
    print("\nUnique values :\n", data.nunique())


In [47]:
def clean(df,fields):
    df = df.dropna()
    ## Convert to datetime
    df.loc[:,fields[0]] = pd.to_datetime(df.loc[:,fields[0]])
    df.loc[:,fields[1]] = pd.to_datetime(df.loc[:,fields[1]])

    ## Convert to travel time
    df.loc[:,'travel_time_diff'] = df[fields[1]] - df[fields[0]]
    df = df.rename(columns={fields[0]:'pickup'})
    
    df = df.drop([fields[1]], axis=1)
    
    df.loc[:,'travel_time'] = df.loc[:,'travel_time_diff'].dt.total_seconds()
    df.loc[:,'travel_time'] = df.loc[:,'travel_time'] / 60
    df = df.drop(['travel_time_diff'], axis=1)

    ## Convert pickup to time of day
    df['pickup_time'] = df['pickup'].dt.hour.astype(int)
    df = df.drop(['pickup'], axis=1)

    # Chomp all decimals
    df.loc[:,'PULocationID'] = df.loc[:,'PULocationID'].astype(int)
    df.loc[:,'DOLocationID'] = df.loc[:,'DOLocationID'].astype(int)
    df.loc[:,'travel_time'] = df.loc[:,'travel_time'].astype(int)

    # remove rows from df that have a longer travel time than the max travel time
    MAX_TRAVEL_TIME = 60*5  # 5 hours
    df = df[df.loc[:,'travel_time'] < MAX_TRAVEL_TIME]
    df = df[df.loc[:,'travel_time'] > 0] 

    #remove row from df if pulocationid is 264 or 265 as they are unknow locations
    df = df[df.loc[:,'PULocationID'] < 264]
    df = df[df.loc[:,'DOLocationID'] < 264]

    return df

In [48]:
def convert_to_coordinates(df):
    # Convert zone ids to coordinates. Remove any rows that have missing coordinates

    lookup = pd.read_csv('taxi_lookup.csv')

    df.loc[:,'OrigLat'] = df.loc[:,'PULocationID'].map(lookup.set_index('LocationID')['lat'])
    df.loc[:,'OrigLon'] = df.loc[:,'PULocationID'].map(lookup.set_index('LocationID')['long'])
    df.loc[:,'DestLat'] = df.loc[:,'DOLocationID'].map(lookup.set_index('LocationID')['lat'])
    df.loc[:,'DestLon'] = df.loc[:,'DOLocationID'].map(lookup.set_index('LocationID')['long'])


    df['Distance'] = ((((df['DestLon'] - df['OrigLon'] )**2) + ((df['DestLat']-df['OrigLat'])**2) )**0.5)

    df = df.drop(['PULocationID', 'DOLocationID'], axis=1)
    return df

In [49]:
def saveData(df,train_filename,test_filename):
    from os.path import exists
    test = df.sample(frac=0.0001)
    df.drop(test.index)
    if(exists(train_filename)):
        df.to_csv(train_filename, mode='a', header=False, index=False)
    else:
        df.to_csv(train_filename, index=False)

    if(exists(test_filename)):
        test.to_csv(test_filename,mode='a',header=False, index=False)
    else:
        test.to_csv(test_filename, index=False)

In [50]:
def dataClean (path, fileEndings ,fields, train_filename,test_filename):
    import datetime

    last = datetime.datetime.now().replace(microsecond=0)
    total = last
    for i in fileEndings:
        print("Reading " + str(i)  , end='')
        df = pd.read_csv(path + str(i) + ".csv", usecols=fields)
        print(".", end = '')
        df = clean(df,fields)
        print(".", end = '')
        df = convert_to_coordinates(df)
        print(". " +" " + str(len(df.index)) +  " rows, Saving -> ", end='')
        saveData(df,train_filename,test_filename)
        print("Saved", end='')
        now = datetime.datetime.now().replace(microsecond=0)
        print(" - " + str(now-last) + " elapsed - total time elapsed " + str(now-total))
        last = now
    print("Total time elapsed: " + str(datetime.datetime.now()-total))


## Yellow

In [51]:
dataClean(path+yellow_path, [1,2,3,4,5,6,7,8,9,10,11,12], yellow_fields , train_filename, test_filename)

Reading 1...  7403443 rows, Saving -> Saved - 0:00:53 elapsed - total time elapsed 0:00:53
Reading 2...  6854375 rows, Saving -> Saved - 0:00:48 elapsed - total time elapsed 0:01:41
Reading 3...  7673438 rows, Saving -> Saved - 0:00:53 elapsed - total time elapsed 0:02:34
Reading 4...  7262022 rows, Saving -> Saved - 0:00:51 elapsed - total time elapsed 0:03:25
Reading 5...  7390685 rows, Saving -> Saved - 0:00:52 elapsed - total time elapsed 0:04:17
Reading 6...  6774429 rows, Saving -> Saved - 0:00:50 elapsed - total time elapsed 0:05:07
Reading 7...  6158872 rows, Saving -> Saved - 0:00:48 elapsed - total time elapsed 0:05:55
Reading 8...  5922087 rows, Saving -> Saved - 0:00:45 elapsed - total time elapsed 0:06:40
Reading 9...  6413960 rows, Saving -> Saved - 0:00:46 elapsed - total time elapsed 0:07:26
Reading 10...  7054853 rows, Saving -> Saved - 0:00:51 elapsed - total time elapsed 0:08:17
Reading 11...  6726880 rows, Saving -> Saved - 0:00:47 elapsed - total time elapsed 0:09:

## High volume

In [52]:
dataClean(path+high_volume_path, list(range(2,13)),high_volume_fields,train_filename, test_filename)

Reading 2...  19549750 rows, Saving -> Saved - 0:02:10 elapsed - total time elapsed 0:02:10
Reading 3...  23131107 rows, Saving -> Saved - 0:02:33 elapsed - total time elapsed 0:04:43
Reading 4...  21051857 rows, Saving -> Saved - 0:02:19 elapsed - total time elapsed 0:07:02
Reading 5...  21617898 rows, Saving -> Saved - 0:02:23 elapsed - total time elapsed 0:09:25
Reading 6...  20298209 rows, Saving -> Saved - 0:02:13 elapsed - total time elapsed 0:11:38
Reading 7...  19642853 rows, Saving -> Saved - 0:02:13 elapsed - total time elapsed 0:13:51
Reading 8...  19447929 rows, Saving -> Saved - 0:02:13 elapsed - total time elapsed 0:16:04
Reading 9...  19373240 rows, Saving -> Saved - 0:02:13 elapsed - total time elapsed 0:18:17
Reading 10...  20434864 rows, Saving -> Saved - 0:02:18 elapsed - total time elapsed 0:20:35
Reading 11...  20930180 rows, Saving -> Saved - 0:02:22 elapsed - total time elapsed 0:22:57
Reading 12...  21530257 rows, Saving -> Saved - 0:02:26 elapsed - total time e

## For hire

In [53]:
dataClean(path+for_hire_path, list(range(1,13)), for_hire_fields, train_filename, test_filename)

Reading 1.

C:\Python39\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Python39\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


..  19830183 rows, Saving -> Saved - 0:02:15 elapsed - total time elapsed 0:02:15
Reading 2...  175984 rows, Saving -> Saved - 0:00:04 elapsed - total time elapsed 0:02:19
Reading 3...  100457 rows, Saving -> Saved - 0:00:03 elapsed - total time elapsed 0:02:22
Reading 4...  225552 rows, Saving -> Saved - 0:00:05 elapsed - total time elapsed 0:02:27
Reading 5...  228956 rows, Saving -> Saved - 0:00:05 elapsed - total time elapsed 0:02:32
Reading 6...  242454 rows, Saving -> Saved - 0:00:05 elapsed - total time elapsed 0:02:37
Reading 7...  257736 rows, Saving -> Saved - 0:00:05 elapsed - total time elapsed 0:02:42
Reading 8...  286283 rows, Saving -> Saved - 0:00:05 elapsed - total time elapsed 0:02:47
Reading 9...  273952 rows, Saving -> Saved - 0:00:04 elapsed - total time elapsed 0:02:51
Reading 10...  308690 rows, Saving -> Saved - 0:00:05 elapsed - total time elapsed 0:02:56
Reading 11...  255725 rows, Saving -> Saved - 0:00:04 elapsed - total time elapsed 0:03:00
Reading 12...  3

## Green

In [54]:
dataClean(path+green_path, list(range(1,13)),green_fields,train_filename, test_filename)

Reading 1...  613054 rows, Saving -> Saved - 0:00:05 elapsed - total time elapsed 0:00:05
Reading 2...  559416 rows, Saving -> Saved - 0:00:04 elapsed - total time elapsed 0:00:09
Reading 3...  584355 rows, Saving -> Saved - 0:00:05 elapsed - total time elapsed 0:00:14
Reading 4...  500092 rows, Saving -> Saved - 0:00:03 elapsed - total time elapsed 0:00:17
Reading 5...  491576 rows, Saving -> Saved - 0:00:04 elapsed - total time elapsed 0:00:21
Reading 6...  458185 rows, Saving -> Saved - 0:00:03 elapsed - total time elapsed 0:00:24
Reading 7...  457789 rows, Saving -> Saved - 0:00:04 elapsed - total time elapsed 0:00:28
Reading 8...  436743 rows, Saving -> Saved - 0:00:03 elapsed - total time elapsed 0:00:31
Reading 9...  436456 rows, Saving -> Saved - 0:00:03 elapsed - total time elapsed 0:00:34
Reading 10...  463979 rows, Saving -> Saved - 0:00:03 elapsed - total time elapsed 0:00:37
Reading 11...  437954 rows, Saving -> Saved - 0:00:03 elapsed - total time elapsed 0:00:40
Reading 